In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
import torchvision

In [4]:
from pathlib import Path

In [5]:
testing_folder = Path('../../.fastai/data/mnist_png/testing')
training_folder = Path('../../.fastai/data/mnist_png/training')

[x for x in testing_folder.iterdir() if x.is_dir()]

[Path('../../.fastai/data/mnist_png/testing/5'),
 Path('../../.fastai/data/mnist_png/testing/3'),
 Path('../../.fastai/data/mnist_png/testing/7'),
 Path('../../.fastai/data/mnist_png/testing/4'),
 Path('../../.fastai/data/mnist_png/testing/2'),
 Path('../../.fastai/data/mnist_png/testing/1'),
 Path('../../.fastai/data/mnist_png/testing/9'),
 Path('../../.fastai/data/mnist_png/testing/0'),
 Path('../../.fastai/data/mnist_png/testing/8'),
 Path('../../.fastai/data/mnist_png/testing/6')]

In [6]:
sorted_testing_folder = testing_folder.ls().sorted()
sorted_training_folder = training_folder.ls().sorted()

sorted_training_folder[0], sorted_testing_folder[3]

(Path('../../.fastai/data/mnist_png/training/0'),
 Path('../../.fastai/data/mnist_png/testing/3'))

In [7]:
threes_tensors = [tensor(Image.open(o)) for o in (training_folder/'3').ls().sorted()]
stacked_threes = torch.stack(threes_tensors).float()/255

sevens_tensors = [tensor(Image.open(o)) for o in (training_folder/'7').ls().sorted()]
stacked_sevens = torch.stack(sevens_tensors).float()/255

#reshape the tensors and concatenate them in a new one
test_tensor = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

stacked_threes.shape, stacked_sevens.shape, test_tensor.shape

(torch.Size([6131, 28, 28]),
 torch.Size([6265, 28, 28]),
 torch.Size([12396, 784]))

In [8]:
def stacked_tensors(path_to_images):
    """A function that returns a stacked Gray Scaled tensor"""
    tensors = [tensor(Image.open(o)) for o in (path_to_images).ls().sorted()]
    # Transforms to grayscale (/255) and Normalizes to mean 0 ((x-0.5)/0.5)
    stacked_tensors = ((torch.stack(tensors).float()/255)-.5)/.5
    #Original line, only converts to grayscale 
    #stacked_tensors = torch.stack(tensors).float()/255

    return stacked_tensors

In [9]:
# Testing the function works as expected
training_stacked_threes = stacked_tensors(sorted_training_folder[3])
training_stacked_sevens = stacked_tensors(sorted_training_folder[7])

training_stacked_threes.shape, training_stacked_sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [10]:
#Corroborate equality of stacked tensors
if torch.equal(stacked_threes, training_stacked_threes) and torch.equal(stacked_sevens, training_stacked_sevens) :
    print("The tensors are equal.")
else:
    print("The tensors are not equal.")

The tensors are equal.


In [11]:
def squash_tensors(list_of_paths):
    listed_tensors = []
    for i in list_of_paths:
        listed_tensors.append(stacked_tensors(i))
    squashed_tensors = torch.cat(listed_tensors).view(-1, 28*28)
    return squashed_tensors

In [12]:
def squash_tensors(list_of_paths):
    squashed_tensors = torch.cat([stacked_tensors(o) for o in list_of_paths]).view(-1, 28*28)
    return squashed_tensors

In [13]:
# Check that the squashed tensors are equal
squashed_threes_and_sevens = squash_tensors([sorted_training_folder[3], sorted_training_folder[7]])

squashed_threes_and_sevens.shape == test_tensor.shape, torch.equal(squashed_threes_and_sevens, test_tensor)

(True, True)

In [14]:
squashed_training_tensors = squash_tensors(sorted_training_folder)

squashed_training_tensors.shape

torch.Size([60000, 784])

In [15]:
valid3s_tensor = torch.stack([tensor(Image.open(o)) for o in (sorted_testing_folder[3]).ls().sorted()]).float()/255
valid7s_tensor = torch.stack([tensor(Image.open(o)) for o in (sorted_testing_folder[7]).ls().sorted()]).float()/255
valid37s_tensor = torch.cat([valid3s_tensor, valid7s_tensor]).view(-1, 28*28)

valid3s_tensor.shape, valid7s_tensor.shape, valid37s_tensor.shape

(torch.Size([1010, 28, 28]),
 torch.Size([1028, 28, 28]),
 torch.Size([2038, 784]))

In [16]:
validation_stacked_threes = stacked_tensors(sorted_testing_folder[3])
validation_stacked_sevens = stacked_tensors(sorted_testing_folder[7])
squashed_valid3and7s_tensor = squash_tensors([sorted_testing_folder[3], sorted_testing_folder[7]])

validation_stacked_threes.shape, validation_stacked_sevens.shape, squashed_valid3and7s_tensor.shape

(torch.Size([1010, 28, 28]),
 torch.Size([1028, 28, 28]),
 torch.Size([2038, 784]))

In [17]:
#Corroborate equality of stacked tensors
if torch.equal(valid3s_tensor, validation_stacked_threes) and torch.equal(valid7s_tensor, validation_stacked_sevens) :
    print("The tensors are equal.")
else:
    print("The tensors are not equal.")

The tensors are equal.


In [18]:
# Check that the squashed tensors are equal
squashed_valid3and7s_tensor.shape == valid37s_tensor.shape, torch.equal(squashed_valid3and7s_tensor, valid37s_tensor)

(True, True)

In [19]:
squashed_validation_tensors = squash_tensors(sorted_testing_folder)

squashed_validation_tensors.shape

torch.Size([10000, 784])

In [20]:
labels_test = tensor([1]*len(threes_tensors) + [0]*len(sevens_tensors)).unsqueeze(1)
labels_test.shape

torch.Size([12396, 1])

In [21]:
# set the validation labels tensor
def getlabels(folders):
    list_of_labels = []
    for o in range(len(folders)):
        list_of_labels = list_of_labels + [o]*len(folders[o].ls())
    labels_tensor = tensor(list_of_labels)
    return labels_tensor

In [22]:
# List comprehension version of the code block above
#start_time = time.time()
#a = [o for o in range(len(sorted_testing_folder)) for _ in range(len(sorted_testing_folder[o].ls()))]
#tensor_a = torch.tensor(a)
#for_loop_time = time.time() - start_time
#for_loop_time

In [23]:
validation_labels_tensor = getlabels(sorted_testing_folder)

training_labels_tensor = getlabels(sorted_training_folder)

In [24]:
validation_dataset = list(zip(squashed_validation_tensors, validation_labels_tensor))

training_dataset = list(zip(squashed_training_tensors, training_labels_tensor))

In [25]:
validation_dataloader = DataLoader(validation_dataset, batch_size = 256, shuffle=True)

training_dataloader = DataLoader(training_dataset, batch_size = 256, shuffle=True)

In [26]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Grayscale(), torchvision.transforms.ToTensor(), torchvision.transforms.Normalize([0.5], [0.5])]
)

testing_dataset = torchvision.datasets.ImageFolder((testing_folder).as_posix(), transform = transform)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=60)

testing_dataloader

In [27]:
dataloaders = {
    "train": training_dataloader,
    "validation": validation_dataloader
}

In [28]:
pytorch_net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 50),
    nn.ReLU(),
    nn.Linear(50,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.LogSoftmax(dim=1))

In [29]:
# nb_epoch is our number of epochs, meaning the number of complete passes through the training dataset.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lr = 1e-2
nb_epoch = 50

device

device(type='cuda', index=0)

In [30]:
optimizer = torch.optim.SGD(pytorch_net.parameters(), lr=lr)

criterion = nn.NLLLoss()

In [31]:
def train_model(model, criterion, optimizer, dataloaders, num_epochs=10):
    #liveloss = PlotLosses() Live training plot generic API
    epochno = 0
    model = model.to(device) # Moves and/or casts the parameters and buffers to device.
    
    for epoch in range(num_epochs): # Number of passes through the entire training & validation datasets
        logs = {}
        for phase in ['train', 'validation']: # First train, then validate
            if phase == 'train':
                model.train() # Set the module in training mode
            else:
                model.eval() # Set the module in evaluation mode

            running_loss = 0.0 # keep track of loss
            running_corrects = 0 # count of carrectly classified inputs

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # Perform Tensor device conversion
                labels = labels.to(device)

                outputs = model(inputs) # forward pass through network
                loss = criterion(outputs, labels) # Calculate loss

                if phase == 'train':
                    optimizer.zero_grad() # Set all previously calculated gradients to 0
                    loss.backward() # Calculate gradients
                    optimizer.step() # Step on the weights using those gradient w -=  gradient(w) * lr

                _, preds = torch.max(outputs, 1) # Get model's predictions
                running_loss += loss.detach() * inputs.size(0) # multiply mean loss by the number of elements
                running_corrects += torch.sum(preds == labels.data) # add number of correct predictions to total

            epoch_loss = running_loss / len(dataloaders[phase].dataset) # get the "mean" loss for the epoch
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset) # Get proportion of correct predictions
            
            # Logging
            prefix = ''
            if phase == 'validation':
                prefix = 'val_'

            logs[prefix + 'log loss'] = epoch_loss.item()
            logs[prefix + 'accuracy'] = epoch_acc.item()
        print('Epoch: ', epochno,' loss: ', epoch_loss.item(), ' accuracy: ', epoch_acc.item())
        epochno += 1
        #liveloss.update(logs) Update logs
        #liveloss.send()  draw, display stuff

In [32]:
train_model(pytorch_net, criterion, optimizer, dataloaders, nb_epoch)

Epoch:  0  loss:  2.2914676666259766  accuracy:  0.16939999163150787
Epoch:  1  loss:  2.2683420181274414  accuracy:  0.21400000154972076
Epoch:  2  loss:  2.219364881515503  accuracy:  0.38589999079704285
Epoch:  3  loss:  2.0795624256134033  accuracy:  0.36890000104904175
Epoch:  4  loss:  1.7697268724441528  accuracy:  0.49619999527931213
Epoch:  5  loss:  1.2530940771102905  accuracy:  0.676099956035614
Epoch:  6  loss:  0.8783851265907288  accuracy:  0.7565999627113342
Epoch:  7  loss:  0.7036246061325073  accuracy:  0.7924999594688416
Epoch:  8  loss:  0.6151942014694214  accuracy:  0.8186999559402466
Epoch:  9  loss:  0.5570011138916016  accuracy:  0.8370999693870544
Epoch:  10  loss:  0.5102318525314331  accuracy:  0.8513000011444092
Epoch:  11  loss:  0.46997132897377014  accuracy:  0.8622999787330627
Epoch:  12  loss:  0.4354182481765747  accuracy:  0.8750999569892883
Epoch:  13  loss:  0.4054580330848694  accuracy:  0.883899986743927
Epoch:  14  loss:  0.38082531094551086  a

In [33]:
torch.save(pytorch_net, 'models/my_own_dc_97pct.pt')

In [33]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Grayscale(), torchvision.transforms.ToTensor(), torchvision.transforms.Normalize([0.5], [0.5])]
)

In [36]:
training_dataset = torchvision.datasets.ImageFolder((training_folder).as_posix(), transform = transform)

In [37]:
batchSize = 64
train_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=batchSize, shuffle=True)

train_dataloader

In [38]:
#testing_dataset = torchvision.datasets.ImageFolder(testing_folder, transform = transform)
#testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=batchSize)
testing_dataset = torchvision.datasets.ImageFolder((testing_folder).as_posix(), transform = transform)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=batchSize)

testing_dataloader

In [39]:
dataloaders = {
    "train": train_dataloader,
    "validation": testing_dataloader
}

In [40]:
pytorch_net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 50),
    nn.ReLU(),
    nn.Linear(50,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.LogSoftmax(dim=1))

In [42]:
# nb_epoch is our number of epochs, meaning the number of complete passes through the training dataset.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lr = 1e-2
nb_epoch = 60

device

device(type='cuda', index=0)

In [43]:
optimizer = torch.optim.SGD(pytorch_net.parameters(), lr=lr)

criterion = nn.NLLLoss()

In [44]:
def train_model(model, criterion, optimizer, dataloaders, num_epochs=10):
    #liveloss = PlotLosses() Live training plot generic API
    model = model.to(device) # Moves and/or casts the parameters and buffers to device.
    
    for epoch in range(num_epochs): # Number of passes through the entire training & validation datasets
        logs = {}
        for phase in ['train', 'validation']: # First train, then validate
            if phase == 'train':
                model.train() # Set the module in training mode
            else:
                model.eval() # Set the module in evaluation mode

            running_loss = 0.0 # keep track of loss
            running_corrects = 0 # count of carrectly classified inputs

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device) # Perform Tensor device conversion
                labels = labels.to(device)

                outputs = model(inputs) # forward pass through network
                loss = criterion(outputs, labels) # Calculate loss

                if phase == 'train':
                    optimizer.zero_grad() # Set all previously calculated gradients to 0
                    loss.backward() # Calculate gradients
                    optimizer.step() # Step on the weights using those gradient w -=  gradient(w) * lr

                _, preds = torch.max(outputs, 1) # Get model's predictions
                running_loss += loss.detach() * inputs.size(0) # multiply mean loss by the number of elements
                running_corrects += torch.sum(preds == labels.data) # add number of correct predictions to total

            epoch_loss = running_loss / len(dataloaders[phase].dataset) # get the "mean" loss for the epoch
            epoch_acc = running_corrects.float() / len(dataloaders[phase].dataset) # Get proportion of correct predictions
            
            # Logging
            prefix = ''
            if phase == 'validation':
                prefix = 'val_'

            logs[prefix + 'log loss'] = epoch_loss.item()
            logs[prefix + 'accuracy'] = epoch_acc.item()
        print('loss: ', epoch_loss.item(), ' accuracy: ', epoch_acc.item())
        
        #liveloss.update(logs) Update logs
        #liveloss.send()  draw, display stuff

In [45]:
train_model(pytorch_net, criterion, optimizer, dataloaders, nb_epoch)

loss:  0.593288242816925  accuracy:  0.8185999989509583
loss:  0.4740932881832123  accuracy:  0.8499999642372131
loss:  0.32599714398384094  accuracy:  0.9025999903678894
loss:  0.30259695649147034  accuracy:  0.9103999733924866
loss:  0.25638362765312195  accuracy:  0.923799991607666
loss:  0.22863425314426422  accuracy:  0.9310999512672424
loss:  0.2064700871706009  accuracy:  0.9378999471664429
loss:  0.18390204012393951  accuracy:  0.9455999732017517
loss:  0.18480242788791656  accuracy:  0.9442999958992004
loss:  0.1646227389574051  accuracy:  0.9505999684333801
loss:  0.1499641239643097  accuracy:  0.9553999900817871
loss:  0.1388562172651291  accuracy:  0.9596999883651733
loss:  0.1377822458744049  accuracy:  0.9587000012397766
loss:  0.12243318557739258  accuracy:  0.9632999897003174
loss:  0.1265191286802292  accuracy:  0.9620999693870544
loss:  0.13990972936153412  accuracy:  0.9577999711036682
loss:  0.11808068305253983  accuracy:  0.9645999670028687
loss:  0.118161506950855

In [21]:
# Save the model
#torch.save(pytorch_net, 'models/my_digit_clasifier_3L_97pct.pt')